In [11]:

import xml.etree.ElementTree as etree
from argparse import ArgumentParser
from pathlib import Path

import fastmri
import h5py
from fastmri.data import transforms
from fastmri.data.mri_data import et_query
from tqdm import tqdm

In [16]:
def save_zero_filled(data_dir, out_dir, which_challenge):
    reconstructions = {}

    for fname in tqdm(list(data_dir.glob("*.h5"))):
        with h5py.File(fname, "r") as hf:
            
            et_root = etree.fromstring(hf["ismrmrd_header"][()])
            masked_kspace = transforms.to_tensor(hf["kspace"][()])

            # extract target image width, height from ismrmrd header
            enc = ["encoding", "encodedSpace", "matrixSize"]
            crop_size = (
                int(et_query(et_root, enc + ["x"])),
                int(et_query(et_root, enc + ["y"])),
            )

            # inverse Fourier Transform to get zero filled solution
            image = fastmri.ifft2c(masked_kspace)

            # check for FLAIR 203
            if image.shape[-2] < crop_size[1]:
                crop_size = (image.shape[-2], image.shape[-2])

            # crop input image
            image = transforms.complex_center_crop(image, crop_size)

            # absolute value
            image = fastmri.complex_abs(image)

            # apply Root-Sum-of-Squares if multicoil data
            if which_challenge == "multicoil":
                image = fastmri.rss(image, dim=1)
            
            plt.imshow(image)
            plt.show()
            return image
            reconstructions[fname.name] = image

#     fastmri.save_reconstructions(reconstructions, out_dir)

    

In [19]:
data_dir  = '/home/svangurp/scratch/samuel/data/knee/train/'
out_dir = 0
which_challenge = 'multicoil'
image = save_zero_filled(data_dir, out_dir, which_challenge)

AttributeError: 'str' object has no attribute 'glob'

<bound method MappingHDF5.keys of <HDF5 file "file1000002.h5" (mode r)>>